This notebook is used for development of helper functions to evantually be included in the feature explorer module.

In [1]:
import pandas as pd

## Load Datasets for testing

In [48]:
# King County
king = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/kc_house_data.csv')

# Water Pumps
pumps = pd.read_csv('Data/pumps.csv')
pumps_target = pd.read_csv('Data/pump_train_labels.csv')
pumps_target = pumps_target['status_group']

# Moores Law
tables = pd.read_html('https://en.wikipedia.org/wiki/Transistor_count', header=0)
moore = tables[0]

# Pedestrian Crossing
crossing = pd.read_csv('Data/daily.csv')

king.shape, pumps.shape, moore.shape, crossing.shape

((21613, 21), (59400, 40), (139, 6), (1063, 9))

In [85]:
king.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Cardinality per Feature Function

In [26]:
def get_cardinality(df, include_numeric=True):
    '''
    Returns: Pandas Series, containing each of the column names as the index values and 
    a count of the unique values within each column as the data.
    
    Parameter: Pandas DataFrame, include_numeric -- should numeric columns be included
    '''
    if include_numeric: 
        cols = df.columns
    else: 
        cols = df.select_dtypes(exclude='number').columns
    uniques = []
    for col in cols:
        uniques.append(df[col].nunique())
    uniques = pd.Series(uniques, cols)
    return uniques

get_cardinality(pumps)[:5]

id               59400
amount_tsh          98
date_recorded      356
funder            1897
gps_height        2428
dtype: int64

### Detect High Cardinality Features Function

In [40]:
def get_high_cardinality(df, threshold=20): 
    ''' 
    Returns a Pandas Series containing a list of the non-numeric columns which contain more than 20 unique values
    
    Parameters: Pandas Dataframe, threshold to determine cutoff point for high cardinality 
    '''
    cardinality = get_cardinality(df, include_numeric=False)
    high_card = cardinality >= 20
    return cardinality[high_card]
    
get_high_cardinality(pumps, 20)

date_recorded      356
funder            1897
installer         2145
wpt_name         37400
subvillage       19287
region              21
lga                125
ward              2092
scheme_name       2696
dtype: int64

### Get Sorted List of Correlations with Specificed Target

In [288]:
def corr_with_target(features, target):
    '''
    Returns a Pandas Series containing the correlations of each feature with the specific target.
    Correlation values will not be returned for non-numeric datatypes. The length of the target Series must 
    equal the dataframe.
    
    If target is non-numeric, it will be ordinal encoded. This may imply an ordering to the target classes that is improper. 
    '''
    if ( target.dtype != ('int' or 'float') ):
        #TODO ordinal encode target
        target = pd.Series( target.factorize()[0] )
        target = target.rename('target')
    # Concat features and target   
    df = pd.concat([features, target], axis=1)
    corrs = df.corr()
    
    # Drop target's correlation with itself
    corrs_target = corrs['target'].sort_values(ascending=False)[1:]
    
    return corrs_target

corr_with_target(king, king['price'])

price            0.213099
sqft_above       0.175123
grade            0.172603
sqft_living      0.168968
sqft_living15    0.156909
bathrooms        0.146990
floors           0.110575
yr_built         0.100722
bedrooms         0.074662
view             0.061059
waterfront       0.036614
long             0.032638
sqft_basement    0.022977
yr_renovated     0.017716
lat              0.003929
id              -0.001906
sqft_lot        -0.009207
sqft_lot15      -0.017667
condition       -0.025234
zipcode         -0.033989
Name: target, dtype: float64

### Detect Suspicious Values

In [215]:
def detect_suspicious(df, custom_vals=[]):
    '''
    Provides a count of suspicious values and returns a dataframe with the counts and the suspicious values.
    In addition, to checking for common suspicious values (e.g., 'N/A') a list of custom suspicious values 
    can be passed.
    '''
    suspicious_vals = ['Not Known', 'Unknown', 'None', 'Not known', 'not known', '-', 
                       'unknown', '##', 'none', '?', '??', '0', -9, -99, 
                       -999, -9999, 9999, 66, 77, 88, "NA", "N A", "N/A", "NA ", " NA", "N /A", 
                       "N / A", " N / A", "N / A ", "na", "n a", "n/a", "na ", " na", "n /a", 
                       "n / a", " a / a", "n / a ", "NULL", "null", "", "\\?", "\\*", "\\."]
    suspicious_vals = suspicious_vals + custom_vals
    cols = df.columns
    suspect_cols = []
    suspect_counts = []
    suspect_vals = []
    for col in cols:
        suspect = (df[col].isin(suspicious_vals)).sum()
        if (suspect > 0):
            suspect_cols.append(col)
            suspect_counts.append(suspect)
            
            # Find intersection of values in suspicious_vals and the column
            vals = list( set(df[col]) & set(suspicious_vals) )
            suspect_vals.append(vals)
    frame = pd.DataFrame({
        'Suspicious Count': suspect_counts,
        'Suspect Values': suspect_vals
    }, index=suspect_cols)
    return frame

detect_suspicious(pumps)

,Suspicious Count,Suspect Values
id,4,"[88, 66, 77, 9999]"
funder,810,"[Unknown, None, Not Known, 0]"
gps_height,97,"[88, 66, 77, -9]"
installer,825,"[Not known, -, unknown, not known, Unknown, 0]"
wpt_name,3588,"[none, Unknown, None, Not Known]"
subvillage,1,[##]
population,20,"[88, 66, 77]"
scheme_management,1,[None]
scheme_name,677,"[none, not known, None]"
management,561,[unknown]


## Outlier Detection 
- Methods adapted from: http://colingorrie.github.io/outlier-detection.html#fn:2

In [203]:
def detect_outliers(df, method='modified_z_score', threshold=3.5):
    if method == 'standard_z_score':
        result = outlier_standard_z_score(df, threshold)
    elif method == 'outlier_iqr':
        result = outlier_iqr(df, threshold)
    else:
        result = outlier_modified_z_score(df, threshold)
    return result

detect_outliers(pumps, threshold=3.5)

,Column,Outlier Count,Outlier Values
0,gps_height,174,"{2560, 2566, 2567, 2568, 2569, 2576, 2585, 261..."
1,longitude,1812,{0.0}
2,region_code,3603,"{80, 90, 99, 60}"
3,district_code,4188,"{33, 67, 43, 13, 80, 30, 53, 23, 60, 62, 63}"
4,population,18031,"{2050, 4100, 8200, 4145, 2100, 2129, 2130, 426..."
5,construction_year,20709,{0}


### Modified Z Score Outlier Detection 

In [221]:
def outlier_modified_z_score(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        median = np.median(df[col])
        median_absolute_deviation = np.median([np.abs(i - median) for i in df[col]])

        modified_z_scores = []
        for i in df[col]:
            if median_absolute_deviation != 0:
                modified_z_scores.append( 0.675 * (i-median) / median_absolute_deviation )
            else: 
                modified_z_scores.append( 0 )


        outliers = np.where(np.abs(modified_z_scores) > z_score_threshold)

        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)

        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
    }, index=outlier_cols)

    return frame

outlier_modified_z_score(pumps, 5)

,Outlier Count,Outlier Values
gps_height,174,"{2560, 2566, 2567, 2568, 2569, 2576, 2585, 261..."
longitude,1812,{0.0}
region_code,3603,"{80, 90, 99, 60}"
district_code,4188,"{33, 67, 43, 13, 80, 30, 53, 23, 60, 62, 63}"
population,18031,"{2050, 4100, 8200, 4145, 2100, 2129, 2130, 426..."
construction_year,20709,{0}


### Z Score Outlier Detection

In [222]:
def outlier_z_score(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        mean = np.mean(df[col])
        stdev = np.std(df[col])
        z_scores = [(i - mean) / stdev for i in df[col]]

        outliers = np.where(np.abs(z_scores) > threshold)
        
        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)


        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
        }, index=outlier_cols)

    return frame

outlier_z_score(pumps, 5)

,Outlier Count,Outlier Values
amount_tsh,105,"{16000.0, 117000.0, 138000.0, 250000.0, 26000...."
longitude,1812,{0.0}
num_private,118,"{131, 136, 141, 150, 280, 668, 672, 160, 300, ..."
district_code,385,"{67, 80, 60, 62, 63}"
population,328,"{3072, 2560, 4100, 3589, 8200, 2569, 2570, 360..."


### IQR Outlier Detection 

In [223]:
def outlier_iqr(df, threshold):
    num_cols = df.select_dtypes(include='number')
    outlier_cols = []
    outlier_counts = []
    outlier_vals = []
    for col in num_cols.columns:
        outliers = []
        quartile_1, quartile_3 = np.percentile(df[col], [25, 75])
        iqr = quartile_3 - quartile_1
        lower = quartile_1 - (iqr * threshold)
        upper = quartile_3 + (iqr * threshold)

        outliers = np.where( (df[col] > upper) | (df[col] < lower) )
        
        outliers_set = set()
        for i in outliers[0]:
            outlier_val = df[col].iloc[i]
            outliers_set.add(outlier_val)

        if outliers[0].size > 0:
            outlier_cols.append(col)
            outlier_counts.append(len(outliers[0]))
            outlier_vals.append(outliers_set)

    frame = pd.DataFrame({
            'Outlier Count': outlier_counts,
            'Outlier Values': outlier_vals
        }, index=outlier_cols)

    return frame

outlier_iqr(pumps, 5)

,Outlier Count,Outlier Values
amount_tsh,10305,"{16000.0, 900.0, 1400.0, 5000.0, 117000.0, 520..."
longitude,1812,{0.0}
num_private,757,"{1, 2, 3, 131, 5, 6, 7, 8, 4, 136, 11, 9, 141,..."
region_code,2578,"{80, 90, 99}"
district_code,3797,"{33, 67, 43, 80, 30, 53, 23, 60, 62, 63}"
population,1071,"{3072, 2560, 2050, 4100, 1540, 3589, 8200, 154..."


# Data Dictionary
- Output a detailed description of a DataFrame which can be used to inform a data scientist's cleaning, imputation, and feature engineering process

In [282]:
def data_dict(df, target):

    null = df.isnull().sum()
    null = pd.DataFrame(null)
    null.columns = ['Null Values']

    unique = df.nunique()
    unique = pd.DataFrame(unique)
    unique.columns = ['Unique Values']

    cardinality = get_cardinality(df)
    cardinality = pd.DataFrame(cardinality)
    cardinality.columns = ['Cardinality']

    datatype = df.dtypes
    datatype = pd.DataFrame(datatype)
    datatype.columns = ['Datatype']

    skew = df.skew()
    skew = pd.DataFrame(skew)
    skew.columns = ['Skew']

    corr_w_target = corr_with_target(df, target)
    corr_w_target = pd.DataFrame(corr_w_target)
    corr_w_target.columns = ['Correlation W/Target']

    suspicious = detect_suspicious(df)
    outlier = detect_outliers(df)
    
    frames = [null, cardinality, datatype, skew, suspicious, corr_w_target, outlier]

    combined = unique
    for frame in frames:
        combined = pd.merge(combined, frame, how='outer', left_index=True, right_index=True)

    counts = ['Suspicious Count', 'Outlier Count']
    combined[counts] = combined[counts].fillna(0)
    vals = ['Suspect Values', 'Outlier Values']
    combined[vals] = combined[vals].fillna('None')
    non_num = ['Skew', 'Correlation W/Target']
    combined[non_num] = combined[non_num].fillna('Non-Numeric')

    col_ordered = ['Datatype', 'Unique Values', 'Null Values', 'Cardinality', 'Correlation W/Target', 'Skew', 'Outlier Count', 'Outlier Values', 'Suspicious Count', 'Suspect Values']
    combined = combined[col_ordered]
    
    return combined

In [289]:
data_dict(pumps, target)

,Datatype,Unique Values,Null Values,Cardinality,Correlation W/Target,Skew,Outlier Count,Outlier Values,Suspicious Count,Suspect Values
amount_tsh,float64,98,0,98,-0.0435335,57.8078,0.0,None,0.0,None
basin,object,9,0,9,Non-Numeric,Non-Numeric,0.0,None,0.0,None
construction_year,int64,55,0,55,-0.0568926,-0.634928,20709.0,{0},0.0,None
date_recorded,object,356,0,356,Non-Numeric,Non-Numeric,0.0,None,0.0,None
district_code,int64,20,0,20,0.0339795,3.96205,4188.0,"{33, 67, 43, 13, 80, 30, 53, 23, 60, 62, 63}",0.0,None
extraction_type,object,18,0,18,Non-Numeric,Non-Numeric,0.0,None,0.0,None
extraction_type_class,object,7,0,7,Non-Numeric,Non-Numeric,0.0,None,0.0,None
extraction_type_group,object,13,0,13,Non-Numeric,Non-Numeric,0.0,None,0.0,None
funder,object,1897,3635,1897,Non-Numeric,Non-Numeric,0.0,None,810.0,"[Unknown, None, Not Known, 0]"
gps_height,int64,2428,0,2428,-0.0964081,0.462402,174.0,"{2560, 2566, 2567, 2568, 2569, 2576, 2585, 261...",97.0,"[88, 66, 77, -9]"


In [290]:
data_dict(king, king['price'])

,Datatype,Unique Values,Null Values,Cardinality,Correlation W/Target,Skew,Outlier Count,Outlier Values,Suspicious Count,Suspect Values
bathrooms,float64,30,0,30,0.14699,0.511108,64.0,"{5.25, 5.5, 5.0, 6.0, 6.75, 5.75, 8.0, 7.5, 7....",0.0,None
bedrooms,int64,13,0,13,0.0746621,1.9743,11.0,"{9, 10, 11, 33}",0.0,None
condition,int64,5,0,5,-0.0252336,1.0328,0.0,None,0.0,None
date,object,372,0,372,Non-Numeric,Non-Numeric,0.0,None,0.0,None
floors,float64,6,0,6,0.110575,0.616177,0.0,None,0.0,None
grade,int64,12,0,12,0.172603,0.771103,14.0,"{1, 13}",0.0,None
id,int64,21436,0,21436,-0.00190627,0.243329,0.0,None,0.0,None
lat,float64,5034,0,5034,0.00392947,-0.48527,0.0,None,0.0,None
long,float64,752,0,752,0.0326382,0.885053,24.0,"{-121.321, -121.402, -121.325, -121.676, -121....",0.0,None
price,float64,4028,0,4028,0.213099,4.02407,929.0,"{1280000.0, 2048000.0, 1749000.0, 2005000.0, 1...",0.0,None


In [291]:
data_dict(crossing, crossing['Total'])

,Datatype,Unique Values,Null Values,Cardinality,Correlation W/Target,Skew,Outlier Count,Outlier Values,Suspicious Count,Suspect Values
AWND,int64,76,0,76,-0.0580965,-32.4998,16.0,"{71, 72, 73, 76, 77, 79, 81, -9999, 88, 95}",8.0,"[88, -9999, 66, 77]"
PRCP,int64,98,0,98,-0.0890183,3.57236,0.0,None,2.0,[66]
SNOW,int64,6,0,6,0.00715107,-16.2322,0.0,None,4.0,[-9999]
SNWD,int64,3,0,3,-0.0115453,29.2068,0.0,None,0.0,None
TMAX,int64,65,0,65,0.303892,0.313213,0.0,None,0.0,None
TMIN,int64,52,0,52,0.265063,-0.251385,0.0,None,0.0,None
Total,float64,942,0,942,0.251208,0.281514,0.0,None,0.0,None
Total_yesterday,float64,942,0,942,0.249881,0.280203,0.0,None,0.0,None
Unnamed: 0,object,1063,0,1063,Non-Numeric,Non-Numeric,0.0,None,0.0,None


In [292]:
data_dict(moore, moore['Transistor count'])

,Datatype,Unique Values,Null Values,Cardinality,Correlation W/Target,Skew,Outlier Count,Outlier Values,Suspicious Count,Suspect Values
Processor,object,139,0,139,Non-Numeric,Non-Numeric,0.0,None,0.0,None
Transistor count,object,130,4,130,Non-Numeric,Non-Numeric,0.0,None,0.0,None
Date of introduction,object,47,0,47,Non-Numeric,Non-Numeric,0.0,None,0.0,None
Designer,object,25,0,25,Non-Numeric,Non-Numeric,0.0,None,0.0,None
Process,object,39,0,39,Non-Numeric,Non-Numeric,0.0,None,0.0,None
Area,object,116,9,116,Non-Numeric,Non-Numeric,0.0,None,0.0,None
